In [0]:
# Read silver data into Gold data frame

df_gold = spark.table("delta_table_silver")

df_gold.limit(5).display()

MOVIES,YEAR,GENRE,RATING,ONE-LINE,STARS,VOTES,RunTime
Blood Red Sky,2021,"Action, Horror, Thriller",6.1,A woman with a mysterious illness is forced into action when a group of terrorists attempt to hijack a transatlantic overnight flight.,"Director: Peter Thorwarth | Stars: Peri Baumeister, Carl Anton Koch, Alexander Scheer, Kais Setti",21062,121
Masters of the Universe: Revelation,2021,"Animation, Action, Adventure",5.0,The war for Eternia begins again in what may be the final battle between He-Man and Skeletor. A new animated series from writer-director Kevin Smith.,"Stars: Chris Wood, Sarah Michelle Gellar, Lena Headey, Mark Hamill",17870,25
The Walking Dead,2010,"Drama, Horror, Thriller",8.2,Sheriff Deputy Rick Grimes wakes up from a coma to learn the world is in ruins and must lead a group of survivors to stay alive.,"Stars: Andrew Lincoln, Norman Reedus, Melissa McBride, Lauren Cohan",885805,44
Rick and Morty,2013,"Animation, Adventure, Comedy",9.2,An animated series that follows the exploits of a super scientist and his not-so-bright grandson.,"Stars: Justin Roiland, Chris Parnell, Spencer Grammer, Sarah Chalke",414849,23
Army of Thieves,2021,"Action, Crime, Horror",7.1,"A prequel, set before the events of Army of the Dead, which focuses on German safecracker Ludwig Dieter leading a group of aspiring thieves on a top secret heist during the early stages of the zombie apocalypse.","Director: Matthias Schweighöfer | Stars: Matthias Schweighöfer, Nathalie Emmanuel, Ruby O. Fee, Stuart Martin",0,60


####Adding the MOVIE_ERA column to the Gold DataFrame<br>

Movies released between 1932 and 1979 are classified as Old.<br>
Movies released between 1980 and 2009 are classified as Middle.<br>
Movies released between 2010 and 2023 are classified as Recent.

In [0]:
from pyspark.sql import functions as F

df_gold = df_gold.withColumn(
    "MOVIE_ERA",
    F.when(F.col("YEAR") <= 1979, "Old")
     .when((F.col("YEAR") >= 1980) & (F.col("YEAR") <= 2009), "Middle")
     .otherwise("Recent")
)

display(df_gold.limit(5))


MOVIES,YEAR,GENRE,RATING,ONE-LINE,STARS,VOTES,RunTime,MOVIE_ERA
Blood Red Sky,2021,"Action, Horror, Thriller",6.1,A woman with a mysterious illness is forced into action when a group of terrorists attempt to hijack a transatlantic overnight flight.,"Director: Peter Thorwarth | Stars: Peri Baumeister, Carl Anton Koch, Alexander Scheer, Kais Setti",21062,121,Recent
Masters of the Universe: Revelation,2021,"Animation, Action, Adventure",5.0,The war for Eternia begins again in what may be the final battle between He-Man and Skeletor. A new animated series from writer-director Kevin Smith.,"Stars: Chris Wood, Sarah Michelle Gellar, Lena Headey, Mark Hamill",17870,25,Recent
The Walking Dead,2010,"Drama, Horror, Thriller",8.2,Sheriff Deputy Rick Grimes wakes up from a coma to learn the world is in ruins and must lead a group of survivors to stay alive.,"Stars: Andrew Lincoln, Norman Reedus, Melissa McBride, Lauren Cohan",885805,44,Recent
Rick and Morty,2013,"Animation, Adventure, Comedy",9.2,An animated series that follows the exploits of a super scientist and his not-so-bright grandson.,"Stars: Justin Roiland, Chris Parnell, Spencer Grammer, Sarah Chalke",414849,23,Recent
Army of Thieves,2021,"Action, Crime, Horror",7.1,"A prequel, set before the events of Army of the Dead, which focuses on German safecracker Ludwig Dieter leading a group of aspiring thieves on a top secret heist during the early stages of the zombie apocalypse.","Director: Matthias Schweighöfer | Stars: Matthias Schweighöfer, Nathalie Emmanuel, Ruby O. Fee, Stuart Martin",0,60,Recent


In [0]:
%sql
select min(rating), max(rating), min(VOTES), max(VOTES) from delta_table_silver where votes != 0

min(rating),max(rating),min(VOTES),max(VOTES)
1.1,9.9,5,1713028


####Adding the POPULARITY_CATEGORY column to the Gold DataFrame

Movies with more than 500,000 votes are classified as Very Popular.<br>
Movies with 100,001 to 500,000 votes are classified as Popular.<br>
Movies with 10,001 to 100,000 votes are classified as Moderate.<br>
Movies with 10,000 votes or fewer are classified as Low Popularity.

In [0]:
from pyspark.sql import functions as F

df_gold = df_gold.withColumn(
    "POPULARITY_CATEGORY",
    F.when(F.col("VOTES") > 500000, "Very Popular")
     .when(F.col("VOTES") > 100000, "Popular")
     .when(F.col("VOTES") > 10000, "Moderate")
     .otherwise("Low Popularity")
)


####Adding the RATING_CATEGORY column to the Gold DataFrame

Movies with ratings 8.5 and above are classified as Excellent.<br>
Movies with ratings between 7.0 and 8.49 are classified as Good.<br>
Movies with ratings between 5.0 and 6.99 are classified as Average.<br>
Movies with ratings below 5.0 are classified as Poor.

In [0]:
df_gold = df_gold.withColumn(
    "RATING_CATEGORY",
    F.when(F.col("RATING") >= 8.5, "Excellent")
     .when(F.col("RATING") >= 7.0, "Good")
     .when(F.col("RATING") >= 5.0, "Average")
     .otherwise("Poor")
)

display(df_gold.limit(5))


MOVIES,YEAR,GENRE,RATING,ONE-LINE,STARS,VOTES,RunTime,MOVIE_ERA,POPULARITY_CATEGORY,RATING_CATEGORY
Blood Red Sky,2021,"Action, Horror, Thriller",6.1,A woman with a mysterious illness is forced into action when a group of terrorists attempt to hijack a transatlantic overnight flight.,"Director: Peter Thorwarth | Stars: Peri Baumeister, Carl Anton Koch, Alexander Scheer, Kais Setti",21062,121,Recent,Moderate,Average
Masters of the Universe: Revelation,2021,"Animation, Action, Adventure",5.0,The war for Eternia begins again in what may be the final battle between He-Man and Skeletor. A new animated series from writer-director Kevin Smith.,"Stars: Chris Wood, Sarah Michelle Gellar, Lena Headey, Mark Hamill",17870,25,Recent,Moderate,Average
The Walking Dead,2010,"Drama, Horror, Thriller",8.2,Sheriff Deputy Rick Grimes wakes up from a coma to learn the world is in ruins and must lead a group of survivors to stay alive.,"Stars: Andrew Lincoln, Norman Reedus, Melissa McBride, Lauren Cohan",885805,44,Recent,Very Popular,Good
Rick and Morty,2013,"Animation, Adventure, Comedy",9.2,An animated series that follows the exploits of a super scientist and his not-so-bright grandson.,"Stars: Justin Roiland, Chris Parnell, Spencer Grammer, Sarah Chalke",414849,23,Recent,Popular,Excellent
Army of Thieves,2021,"Action, Crime, Horror",7.1,"A prequel, set before the events of Army of the Dead, which focuses on German safecracker Ludwig Dieter leading a group of aspiring thieves on a top secret heist during the early stages of the zombie apocalypse.","Director: Matthias Schweighöfer | Stars: Matthias Schweighöfer, Nathalie Emmanuel, Ruby O. Fee, Stuart Martin",0,60,Recent,Low Popularity,Good


In [0]:
''' Gold DataFrame was written to the ADLS Gold layer in Delta format using overwrite mode.'''

ADLS_Gold_path = "/mnt/ADLSmount/gold/movies_dataset/"

df_gold.write.format("delta") \
    .mode("overwrite")\
    .save(ADLS_Gold_path)

#####Gold dataset is present in ADLS in Parquet format
![Gold Layer in ADLS](https://adb-582130891499017.17.azuredatabricks.net/files/tables/ADLSsnipGold.png)

In [0]:
spark.sql("""
    CREATE TABLE IF NOT EXISTS delta_table_gold
    USING DELTA
    LOCATION '{}'
""".format(ADLS_Gold_path))

DataFrame[]

In [0]:
%sql
SELECT * FROM delta_table_gold where LIMIT 5;

MOVIES,YEAR,GENRE,RATING,ONE-LINE,STARS,VOTES,RunTime,MOVIE_ERA,POPULARITY_CATEGORY,RATING_CATEGORY
Blood Red Sky,2021,"Action, Horror, Thriller",6.1,A woman with a mysterious illness is forced into action when a group of terrorists attempt to hijack a transatlantic overnight flight.,"Director: Peter Thorwarth | Stars: Peri Baumeister, Carl Anton Koch, Alexander Scheer, Kais Setti",21062,121,Recent,Moderate,Average
Masters of the Universe: Revelation,2021,"Animation, Action, Adventure",5.0,The war for Eternia begins again in what may be the final battle between He-Man and Skeletor. A new animated series from writer-director Kevin Smith.,"Stars: Chris Wood, Sarah Michelle Gellar, Lena Headey, Mark Hamill",17870,25,Recent,Moderate,Average
The Walking Dead,2010,"Drama, Horror, Thriller",8.2,Sheriff Deputy Rick Grimes wakes up from a coma to learn the world is in ruins and must lead a group of survivors to stay alive.,"Stars: Andrew Lincoln, Norman Reedus, Melissa McBride, Lauren Cohan",885805,44,Recent,Very Popular,Good
Rick and Morty,2013,"Animation, Adventure, Comedy",9.2,An animated series that follows the exploits of a super scientist and his not-so-bright grandson.,"Stars: Justin Roiland, Chris Parnell, Spencer Grammer, Sarah Chalke",414849,23,Recent,Popular,Excellent
Army of Thieves,2021,"Action, Crime, Horror",7.1,"A prequel, set before the events of Army of the Dead, which focuses on German safecracker Ludwig Dieter leading a group of aspiring thieves on a top secret heist during the early stages of the zombie apocalypse.","Director: Matthias Schweighöfer | Stars: Matthias Schweighöfer, Nathalie Emmanuel, Ruby O. Fee, Stuart Martin",0,60,Recent,Low Popularity,Good
